In [40]:
fourXYZ = 'GGACTGGCCCTTAAGTTAGGCATCAGTGCCGACGCCGCTTATAAACTTGCTAAAGCAGCCTACTATTCAGGAACTACCGTTGAAGAAGCCTACAAATTGGCTCTTAAGTTGGGAATATCCGCTGATGCGGCTTATAAGCTTGCGGAAGCAGCCTACTATAGTGGAACCACTGTGGAAGAGGCCTATAAACTTGCATTAAAACTGGGGATCTCCGCTGATGCTGCATACAAATTAGCAAAAGCGGCGTACTACTCGGGCACTACCGTTGAAGAAGCATATAAGTTAGCACTGAAACTCGGGATCAGCGCGGACGCGGCTTACAAACTTGCAAAGGCGGCCTACTACTCGGGAACGACAGTGGAAGAGGCATATAAGCTTGCACTTAAACTGGGCATTAGTGCCGACGCAGCGTACAAGTTAGCCAAAGCCGCATATTATTCAGGTACGACTGTTGAAGAGGCATACAAATTAGCACTGAAGCTCGGCATATCCGCGGACGCCGCGTATAAACTGGCTAAAGCTGCTTACTATAGCGGCACGACAGTAGAGGAAGCTTACAAATTGGCCCTCAAGTTAGGTATCTCGGCGGATGCCGCGTATAAACTTGCAAAAGCGGCGTATTACTCAGGCACAACCGTTGAGGAAGCTTATAAATTGGCATTGAAATTAGGCATAAGTGCCGACGCTGCTTACAAACTGGCCAAGGCCGCGTACTACTCGGGAACAACGGTCGAAGAGGCATACAAGCTGGCATTAAAGTTAGGCATTAGCGCGGATGCTGCTTATAAATTGGCCAAAGCAGCATACTATTCTGGGACAACTGTCGAGGAAGCGTACAAGCTGGCACTGAAGTTGGGC'
eightE2Q = 'ATGAGCGAAGAACGCGTGGAAGAAGAACTGCGCCGCCTGGTGGAAGAAGAAGCGGAAGCGCGCGGCATTAGCCGCCGCGAAACCCTGGAACGCCTGAGCCTGCTGCTGTTTAGCATGCAGCTGGAAAAACTGGTGAAAGAAGAAGCGGAAGCGCGCGGCGTGAGCGTGGAAACCATTCGCGAAGAACTGGAACGCGAAGTGGATGAACGCCTGCGCGAACTGCGCGAAGAAGGCATTAGCCGCCGCGAAACCCTGGAACGCCTGAGCCTGCTGCTGTTTAGCATGCAGCTGGAAAAACTGGTGAAAGAAGAAGCGGAAGCGCGCGGCGTGAGCGTGGAAACCATTCGCGAAGAACTGGAACGCGAAGTGGATGAACGCCTGCGCGAACTGCGCGAAGAAGGCATTAGCCGCCGCGAAACCCTGGAACGCCTGAGCCTGCTGCTGTTTAGCATGCAGCTGGAAAAACTGGTGAAAGAAGAAGCGGAAGCGCGCGGCGTGAGCGTGGAAGAAATTCGCGAAGAACTGCGCCGCGAAGTGGATGAACGCCTGCGCGAACTGCGCGAAGAAGGCATTAGCCGCCGCGAAACCCTGGAACGCCTGAGCCTGCTGCTGTTTAGCATGCAGCTGGAAAAACTGGTGAAAGAAGAAGCGGAAGCGCGCGGCGTGAGCGTGGAAGAAATTCGCGAAGAACTGCGCCGCGAAGTGGATGAACGCCTGCGCGAACTGCGCGAAGAAGGCATTAGCCGCCGCGAAACCCTGGAACGCCTGAGCCTGCTGCTGTTTAGCATGCAGCTGGAAAAACTGGTGAAAGAAGAAGCGGAAGCGCGCGGCGTGAGCGTGGAAGAAATTCGCGAAGAACTGCGCCGCGAAGTGGAAGAACGCCTGCGCGAACTGCGCGAAGAAGGCGAA'
sevenSQ3 = 'GGCAGCAGCATGGGTTCAGACGAGCAACGACGTGAGCTTGAGGAGAAGATAAAGTTCAAGCTCGCTGAGTTGGCTTCAAAGTCAGAGGAAGAGCGGAAGGAGATCAAGCTCAGAGTTATAGCCTACGTTTTAGTTCAATTAGAGGACTTACAAAAGTTGTTATCTGACGAGCAACGTCGGGAGTTAGAGGAGAAGATAAAGTTCAAGCTTGCTGAGCTCGCCTCAAAGTCTGAGGAAGAGCGTAAGGAAATCAAGCTCAGAGTAATAGCATACGTTTTAGTTCAACTTGAGGACTTACAAAAGCTTTTATCTGACGAGCAACGTCGTGAGTTGGAAGAGAAGATAAAGTTCAAGTTAGCAGAGCTTGCTAGTAAGTCAGAAGAGGAGAGAAAGGAGATCAAGTTACGGGTTATAGCATACGTCTTGGTCCAACTTGAGGACTTACAAAAGTTACTCTCC'

In [66]:
from PIL import Image
import numpy as np

def bend_to_circle(img, inner_radius=200, outer_radius=600):
    """
    Warp a horizontal image into a ring shape using polar transform.
    - inner_radius: radius of the hole in the middle
    - outer_radius: outer radius of the ring
    """
    img = img.convert("RGBA")
    w, h = img.size

    # Output image size
    out_r = outer_radius
    out_size = out_r * 2
    out = Image.new("RGBA", (out_size, out_size), (255, 255, 255, 0))

    # Convert to numpy for speed
    src = np.array(img)
    dst = np.zeros((out_size, out_size, 4), dtype=np.uint8)

    center = out_size // 2
    ring_thickness = outer_radius - inner_radius

    for y in range(out_size):
        for x in range(out_size):
            dx = x - center
            dy = y - center
            r = np.sqrt(dx*dx + dy*dy)
            if inner_radius <= r < outer_radius:
                theta = np.arctan2(dy, dx)
                if theta < 0:
                    theta += 2 * np.pi
                # Map radius → vertical pixel, angle → horizontal pixel
                src_x = int(theta / (2 * np.pi) * w)
                src_y = int((r - inner_radius) / ring_thickness * h)
                if 0 <= src_x < w and 0 <= src_y < h:
                    dst[y, x] = src[src_y, src_x]

    return Image.fromarray(dst, "RGBA")


def create_concentric_rings(img_path, innermost_radius=100, ring_height=50, margin=10):
    """
    Create concentric rings from a long horizontal image.
    
    Parameters:
    - img_path: path to the long horizontal image
    - innermost_radius: radius of the innermost ring's inner edge
    - ring_height: height of each ring (thickness)
    - margin: spacing between rings
    
    Returns:
    - composite image with all concentric rings
    """
    # Load the long horizontal image
    long_img = Image.open(img_path).convert("RGBA")
    img_width, img_height = long_img.size
    
    # Calculate how many rings we can make
    rings = []
    current_x = 0
    current_inner_radius = innermost_radius
    
    while current_x < img_width:
        inner_r = current_inner_radius
        outer_r = inner_r + ring_height
        
        # Calculate circumference at the middle of this ring
        mid_radius = (inner_r + outer_r) / 2
        circumference = 2 * np.pi * mid_radius
        
        # Width of strip needed for this ring
        strip_width = int(circumference)
        
        # Check if we have enough image left
        if current_x + strip_width > img_width:
            strip_width = img_width - current_x
            if strip_width < 10:  # Skip if too small
                break
        
        # Extract the strip
        strip = long_img.crop((current_x, 0, current_x + strip_width, img_height))
        
        # Bend to circle
        ring = bend_to_circle(strip, inner_radius=inner_r, outer_radius=outer_r)
        rings.append(ring)
        
        # Move to next ring
        current_x += strip_width
        current_inner_radius = outer_r + margin
    
    # Composite all rings onto one image
    if not rings:
        return None
    
    # Find the size needed (based on outermost ring)
    max_size = max(ring.size[0] for ring in rings)
    final_img = Image.new("RGBA", (max_size, max_size), (255, 255, 255, 0))
    
    # Paste each ring centered
    for ring in rings:
        offset = (max_size - ring.size[0]) // 2
        final_img.paste(ring, (offset, offset), ring)
    
    return final_img


# Example usage
result = create_concentric_rings(
    "test_dna_font.png",
    innermost_radius=300,
    ring_height=500,
    margin=10
)

if result:
    result.save("dna_concentric.png")
    print("Saved dna_concentric.png")
else:
    print("No rings created - image may be too short")


Saved dna_concentric.png


In [ ]:
# from PIL import Image, ImageDraw
# import math
# import os

# # --- SETTINGS ---
# glyph_folder = "DNA_FONT"   # folder with your glyph PNGs
# canvas_size = 1000        # output size
# radius = 350              # circle radius for glyph placement
# output_file = "circle_glyphs.png"

# # --- SETUP CANVAS ---
# canvas = Image.new("RGBA", (canvas_size, canvas_size), (0, 0, 0, 0))
# center = canvas_size // 2

# # --- LOAD GLYPHS ---
# glyph_files = sorted([f for f in os.listdir(glyph_folder) if f.endswith(".png")])
# n = len(glyph_files)

# # --- PLACE EACH GLYPH ---
# for i, glyph_file in enumerate(glyph_files):
#     glyph = Image.open(os.path.join(glyph_folder, glyph_file)).convert("RGBA")

#     # angle for this glyph
#     angle = (2 * math.pi * i) / n

#     # position on the circle
#     x = center + radius * math.cos(angle)
#     y = center + radius * math.sin(angle)

#     # rotate glyph so it points outward
#     degrees = math.degrees(angle)  - 90  # -90 makes them face outward
#     rotated = glyph.rotate(-degrees, expand=True)

#     # center glyph at (x,y)
#     gx, gy = rotated.size
#     canvas.paste(rotated, (int(x - gx/2), int(y - gy/2)), rotated)

# # --- SAVE RESULT ---
# canvas.save(output_file)
# print(f"Saved {output_file}")


Saved circle_glyphs.png


In [ ]:
# from PIL import Image
# import numpy as np

# def bend_to_circle_content_aware(img, inner_radius=200, outer_radius=600):
#     """
#     Warp a horizontal image into a circle while ignoring transparent regions,
#     so the top and bottom of the content are proportionally preserved.
#     """
#     img = img.convert("RGBA")
#     w, h = img.size

#     # Compute which rows have content
#     src_array = np.array(img)
#     # Any pixel with alpha > 0 counts as content
#     row_content = (src_array[:, :, 3] > 0).astype(np.float32)
#     # Sum across columns to get content per row
#     row_sum = row_content.sum(axis=1)
#     # Avoid division by zero
#     if row_sum.sum() == 0:
#         raise ValueError("Image has no visible content!")

#     # Compute cumulative normalized content length (CDF)
#     cumulative = np.cumsum(row_sum)
#     cumulative /= cumulative[-1]  # normalize to 0..1

#     # Output image
#     out_r = outer_radius
#     out_size = out_r * 2
#     dst = np.zeros((out_size, out_size, 4), dtype=np.uint8)
#     center = out_size // 2
#     ring_thickness = outer_radius - inner_radius

#     for y in range(out_size):
#         for x in range(out_size):
#             dx = x - center
#             dy = y - center
#             r = np.sqrt(dx*dx + dy*dy)
#             if inner_radius <= r < outer_radius:
#                 theta = np.arctan2(dy, dx)
#                 if theta < 0:
#                     theta += 2 * np.pi
#                 # Horizontal mapping: angle -> x
#                 src_x = int(theta / (2 * np.pi) * w)
#                 # Vertical mapping: map radius to compressed content coordinate
#                 t = (r - inner_radius) / ring_thickness
#                 # Find the src_y corresponding to cumulative t
#                 src_y = np.searchsorted(cumulative, t)
#                 src_y = min(h-1, src_y)
#                 dst[y, x] = src_array[src_y, src_x]

#     return Image.fromarray(dst, "RGBA")

# # Example usage
# strip = Image.open("test_dna_font.png")
# circular = bend_to_circle_content_aware(strip, inner_radius=200, outer_radius=600)
# circular.save("dna_circle_content.png")
# print("Saved dna_circle_content.png")


Saved dna_circle_content.png


In [61]:
from PIL import Image
import numpy as np

outer_radius=600

def bend_to_circle(img, inner_radius=200, outer_radius=600):
    """
    Warp a horizontal image into a ring shape using polar transform.
    - inner_radius: radius of the hole in the middle
    - outer_radius: outer radius of the ring
    """
    img = img.convert("RGBA")
    w, h = img.size

    out_r = outer_radius
    out_size = out_r * 2
    dst = np.zeros((out_size, out_size, 4), dtype=np.uint8)
    src = np.array(img)
    center = out_size // 2
    ring_thickness = outer_radius - inner_radius

    for y in range(out_size):
        for x in range(out_size):
            dx = x - center
            dy = y - center
            r = np.sqrt(dx*dx + dy*dy)
            if inner_radius <= r < outer_radius:
                theta = np.arctan2(dy, dx)
                if theta < 0:
                    theta += 2 * np.pi
                src_x = int(theta / (2 * np.pi) * w)
                src_y = int((r - inner_radius) / ring_thickness * h)
                if 0 <= src_x < w and 0 <= src_y < h:
                    dst[y, x] = src[src_y, src_x]

    return Image.fromarray(dst, "RGBA")


def concentric_rings(img, start_radius=100, ring_width=50, margin=10):
    """
    Create concentric rings from a long horizontal image.
    
    Parameters:
    - img: horizontal strip image (PIL.Image)
    - start_radius: radius of the innermost ring
    - ring_width: width of each ring
    - margin: space between rings
    """
    img = img.convert("RGBA")
    w, h = img.size

    # Determine how many rings can fit
    max_radius = start_radius + ring_width
    num_rings = 0
    total_height = start_radius
    while total_height + ring_width <= max_radius * 1000:  # arbitrary large outer limit
        total_height += ring_width + margin
        num_rings += 1

    # Alternative: we can just split the image into num_rings proportional to ring size
    # Let's instead calculate number of rings from the image length
    # Here we assume ring width + margin per ring
    total_rings = (img.width // h) or 1  # naive approximation
    # More flexible: let's compute dynamically
    # We'll just split the strip into chunks proportional to ring radius later

    # Calculate outer radius of each ring
    rings = []
    current_inner = start_radius
    remaining_width = img.width
    total_rings = 0
    # We'll first decide how many rings fit given the height
    while current_inner + ring_width <= 2000:  # arbitrary max outer radius
        total_rings += 1
        current_inner += ring_width + margin

    # Actually, let's do a more practical version:
    # We'll create rings until we run out of horizontal pixels in the source image

    rings = []
    radii = []
    inner = start_radius
    src_x_start = 0
    while src_x_start < w:
        outer = inner + ring_width
        # Determine how many horizontal pixels should go into this ring
        # We'll map proportionally: area of ring vs remaining area of image
        circumference = 2 * np.pi * ((inner + outer) / 2)
        pixels_for_ring = int(circumference)  # proportional to circumference
        src_x_end = min(w, src_x_start + pixels_for_ring)
        chunk = img.crop((src_x_start, 0, src_x_end, h))
        rings.append((chunk, inner, outer))
        radii.append((inner, outer))
        src_x_start = src_x_end
        inner = outer + margin
        if inner >= outer_radius:
            break

    # Determine output image size
    final_out_r = inner
    out_size = final_out_r * 2
    out_image = Image.new("RGBA", (out_size, out_size), (255, 255, 255, 0))

    # Draw rings
    for chunk, inner_r, outer_r in rings:
        ring_img = bend_to_circle(chunk, inner_r, outer_r)
        # Paste the ring onto the final image
        out_image = Image.alpha_composite(out_image, ring_img)

    return out_image


strip = Image.open("test_dna_font.png")
result = concentric_rings(strip, start_radius=200, ring_width=50, margin=10)
result.save("dna_concentric.png")
print("Saved dna_concentric.png")


ValueError: images do not match

In [57]:
from PIL import Image
import numpy as np

def bend_to_circle(img, inner_radius=200, outer_radius=600):
    """
    Warp a horizontal image into a ring shape using polar transform.
    - inner_radius: radius of the hole in the middle
    - outer_radius: outer radius of the ring
    """
    img = img.convert("RGBA")
    w, h = img.size

    # Output image size
    out_r = outer_radius
    out_size = out_r * 2
    out = Image.new("RGBA", (out_size, out_size), (255, 255, 255, 0))

    # Convert to numpy for speed
    src = np.array(img)
    dst = np.zeros((out_size, out_size, 4), dtype=np.uint8)

    center = out_size // 2
    ring_thickness = outer_radius - inner_radius

    for y in range(out_size):
        for x in range(out_size):
            dx = x - center
            dy = y - center
            r = np.sqrt(dx*dx + dy*dy)
            if inner_radius <= r < outer_radius:
                theta = np.arctan2(dy, dx)
                if theta < 0:
                    theta += 2 * np.pi
                # Map radius → vertical pixel, angle → horizontal pixel
                src_x = int(theta / (2 * np.pi) * w)
                src_y = int((r - inner_radius) / ring_thickness * h)
                if 0 <= src_x < w and 0 <= src_y < h:
                    dst[y, x] = src[src_y, src_x]

    return Image.fromarray(dst, "RGBA")

# # Example usage
# if __name__ == "__main__":
#     # Load your pre-rendered horizontal DNA strip
#     strip = Image.open("dna_font.png")  # your existing straight rendering
#     circular = bend_to_circle(strip, inner_radius=200, outer_radius=600)
#     circular.save("dna_circle.png")
#     print("Saved dna_circle.png")


In [58]:
# Load your pre-rendered horizontal DNA strip
strip = Image.open("test_dna_font.png")  # your existing straight rendering
circular = bend_to_circle(strip, inner_radius=200, outer_radius=600)
circular.save("dna_circle.png")
print("Saved dna_circle.png")


Saved dna_circle.png


In [43]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
render_dna_sequence_concentric(fourXYZ, "4YXZ_dna_font_circle.png")

Saved: 4XYZ_dna_font.png


In [ ]:
render_dna_sequence_concentric(eightE2Q, "8ETQ_dna_font_circle.png")

Saved: 8E2Q_dna_font.png


In [ ]:
render_dna_sequence_concentric(sevenSQ3, "7SQ3_dna_font_circle.png")

Saved: 7SQ3_dna_font.png
